Load packages

In [69]:
using DataFrames, CSV, Dates, CategoricalArrays, Colors, ColorSchemes, PlotlyJS

Read in data. Adding a semicolon at the end suppresses the long output.

In [70]:
ppa_price = CSV.read("/Users/katiepress/Desktop/Rproj/Tidy Tuesday/ppa_price.csv", DataFrame);

Check out the data types. 

In [71]:
eltype.(eachcol(ppa_price))

12-element Vector{DataType}:
 String31
 Float64
 String31
 String31
 String31
 String31
 String31
 String31
 String31
 String31
 String31
 String31

Check out the first few rows of data.

In [72]:
first(ppa_price, 6)

,PPA Execution Date,Capacity (MW),CAISO,West (non-ISO),MISO
,String31,Float64,String31,String31,String31
1,2006-09-01T00:00:00Z,7.0,NA,239.8734302320258,NA
2,2007-06-25T00:00:00Z,5.0,220.6314143578111,NA,NA
3,2008-07-01T00:00:00Z,550.0,150.73040524305304,NA,NA
4,2008-07-23T00:00:00Z,210.0,131.7706901652578,NA,NA
5,2008-12-19T00:00:00Z,10.0,168.7879318708772,NA,NA
6,2009-02-23T00:00:00Z,30.0,NA,132.09678867228317,NA


Convert the data from wide to long. In Julia this is done using stack() which seems similar to gather() in tidyverse. 

In [73]:
ppa_price_long = stack(ppa_price, Not(1:2));
first(ppa_price_long, 6)

,PPA Execution Date,Capacity (MW),variable,value
,String31,Float64,String,String31
1,2006-09-01T00:00:00Z,7.0,CAISO,NA
2,2007-06-25T00:00:00Z,5.0,CAISO,220.6314143578111
3,2008-07-01T00:00:00Z,550.0,CAISO,150.73040524305304
4,2008-07-23T00:00:00Z,210.0,CAISO,131.7706901652578
5,2008-12-19T00:00:00Z,10.0,CAISO,168.7879318708772
6,2009-02-23T00:00:00Z,30.0,CAISO,NA


Need to rename the columns. Julia has a rename() function that looks pretty standard. Interesting though, if you use an exclamation point in the function it will alter the dataframe in place, so you don't have to assign it back to itself or to a new object. 

In [74]:
rename!(ppa_price_long, [:ppa_execution_date, :capacity_mw, :region, :ppa_price]);
first(ppa_price_long, 6)

,ppa_execution_date,capacity_mw,region,ppa_price
,String31,Float64,String,String31
1,2006-09-01T00:00:00Z,7.0,CAISO,NA
2,2007-06-25T00:00:00Z,5.0,CAISO,220.6314143578111
3,2008-07-01T00:00:00Z,550.0,CAISO,150.73040524305304
4,2008-07-23T00:00:00Z,210.0,CAISO,131.7706901652578
5,2008-12-19T00:00:00Z,10.0,CAISO,168.7879318708772
6,2009-02-23T00:00:00Z,30.0,CAISO,NA


Julia has this "missing" type that is supposed to be like NULL in SQL and NA in R (according to the documentation). First change the NAs to missing. For some reason it wouldn't let me use the exact same code below with replace!(), so I had to assign it back to itself, wonder why that is. 

In [75]:
ppa_price_long.ppa_price = replace(ppa_price_long.ppa_price, "NA" => missing);

Get rid of the missings in the ppa_price column and check to make sure there are 333 rows left. And it worked! 

In [76]:
subset!(ppa_price_long, :ppa_price => ByRow(!ismissing));
nrow(ppa_price_long)

333

Convert the region variable to a factor. Looks like Julia has the CategoricalArrays package which seems similar to Categorical in Python or factors in R. Looks like I can transform it in place, and the  levels are assigned alphabetically. 

In [77]:
transform!(ppa_price_long, :region => categorical);
levels(ppa_price_long.region_categorical)

10-element Vector{String}:
 "CAISO"
 "ERCOT"
 "Hawaii"
 "ISO-NE"
 "MISO"
 "NYISO"
 "PJM"
 "SPP"
 "Southeast (non-ISO)"
 "West (non-ISO)"

Suppose I'd better change the ppa_price from string to numeric as well. This is something weird in Julia, the column type is both string and missing? 

In [78]:
eltype.(eachcol(ppa_price_long))

5-element Vector{Type}:
 String31
 Float64
 String
 Union{Missing, String31}
 CategoricalValue{String, UInt32}

Guessing if I drop the missings (there aren't any) the column type will be fixed? (It worked!)

In [79]:
dropmissing!(ppa_price_long, :ppa_price);
eltype.(eachcol(ppa_price_long))

5-element Vector{DataType}:
 String31
 Float64
 String
 String31
 CategoricalValue{String, UInt32}

Now finally I can convert it to numeric. Since it's in a dataframe column I'll use transform() again since I would usually do this with dplyr functions in R as well. The commented-out code is another way I found it could be done using map, but I'm just going to stick with using transform for now until I get used to the syntax. 

In [80]:
#map(x->parse(Float64, x), ppa_price_long.ppa_price)
transform!(ppa_price_long, :ppa_price => ByRow(x -> parse(Float64, x)) => :ppa_price);
first(ppa_price_long)

,ppa_execution_date,capacity_mw,region,ppa_price,region_categorical
,String31,Float64,String,Float64,Cat…
1,2007-06-25T00:00:00Z,5.0,CAISO,220.631,CAISO


Create a new date variable. Was having a bit of a hard time figuring out the datetime manipulation in Julia, I'm converting it to a string first. Note that transform() is a bit like dplyr's mutate(). 

In [81]:
transform!(ppa_price_long, :ppa_execution_date => ByRow(x -> SubString(x, 1, 10)) => :ppa_date);
first(ppa_price_long, 6)

,ppa_execution_date,capacity_mw,region,ppa_price,region_categorical,ppa_date
,String31,Float64,String,Float64,Cat…,SubStrin…
1,2007-06-25T00:00:00Z,5.0,CAISO,220.631,CAISO,2007-06-25
2,2008-07-01T00:00:00Z,550.0,CAISO,150.73,CAISO,2008-07-01
3,2008-07-23T00:00:00Z,210.0,CAISO,131.771,CAISO,2008-07-23
4,2008-12-19T00:00:00Z,10.0,CAISO,168.788,CAISO,2008-12-19
5,2009-05-08T00:00:00Z,230.0,CAISO,142.783,CAISO,2009-05-08
6,2009-06-22T00:00:00Z,48.0,CAISO,166.372,CAISO,2009-06-22


Now to format it as an actual date. Apparently it will be really slow if you don't define a date format first because Julia will create a new format for each individual line? Also this syntax looks really simple but it took me a minute, I tried a lot more complicated things first. 

In [82]:
myFormat = Dates.DateFormat("yyyy-mm-dd")
ppa_price_long.ppa_date = Date.(ppa_price_long.ppa_date, myFormat);
first(ppa_price_long, 6)


,ppa_execution_date,capacity_mw,region,ppa_price,region_categorical,ppa_date
,String31,Float64,String,Float64,Cat…,Date
1,2007-06-25T00:00:00Z,5.0,CAISO,220.631,CAISO,2007-06-25
2,2008-07-01T00:00:00Z,550.0,CAISO,150.73,CAISO,2008-07-01
3,2008-07-23T00:00:00Z,210.0,CAISO,131.771,CAISO,2008-07-23
4,2008-12-19T00:00:00Z,10.0,CAISO,168.788,CAISO,2008-12-19
5,2009-05-08T00:00:00Z,230.0,CAISO,142.783,CAISO,2009-05-08
6,2009-06-22T00:00:00Z,48.0,CAISO,166.372,CAISO,2009-06-22


Now I actually want to make a year column as well that can be used for aggregation for the bar chart. Fun thing I found in the documentation - with the code below, if I use Dates.Year (uppercase Y) it returns a period object. If using lowercase, it returns an Int which is probably more what I want. 

In [83]:
transform!(ppa_price_long, :ppa_date => ByRow(x -> Dates.year(x)) => :ppa_year);
first(ppa_price_long, 6)

,ppa_execution_date,capacity_mw,region,ppa_price,region_categorical,ppa_date
,String31,Float64,String,Float64,Cat…,Date
1,2007-06-25T00:00:00Z,5.0,CAISO,220.631,CAISO,2007-06-25
2,2008-07-01T00:00:00Z,550.0,CAISO,150.73,CAISO,2008-07-01
3,2008-07-23T00:00:00Z,210.0,CAISO,131.771,CAISO,2008-07-23
4,2008-12-19T00:00:00Z,10.0,CAISO,168.788,CAISO,2008-12-19
5,2009-05-08T00:00:00Z,230.0,CAISO,142.783,CAISO,2009-05-08
6,2009-06-22T00:00:00Z,48.0,CAISO,166.372,CAISO,2009-06-22


Plotting in Julia's gr() wasn't really ideal so I decided to just go straight to plotly (I tried several other things first actually). I was able to find a solution on stackoverflow for mapping the colors but it's not ideal, I don't understand why there is no ability to easily use a colormap like there is with Python. 

In [84]:
mycolorpal = Dict(
    "CAISO" => "#4E79A7", 
    "ERCOT" => "#F28E2B", 
    "Hawaii" => "#E15759", 
    "ISO-NE" => "#76B7B2",
    "MISO" => "#59A14F", 
    "NYISO" => "#BAB0AC", 
    "PJM" => "#9C755F", 
    "Southeast (non-ISO)" => "#FF9DA7",
    "SPP" => "#B07AA1", 
    "West (non-ISO)" => "#EDC948");


In [85]:

plot([scatter(
    subdf,
    x=:ppa_date,
    y=:ppa_price,
    color=:region_categorical,
    mode="markers",
    marker=attr(size=:capacity_mw, sizemode="area", symbol="circle-open"),
    marker_color = mycolorpal[subdf[1, :region_categorical]],
    marker_line_width = 2,
    name=subdf[1, :region_categorical],
    labels = Dict(
        :region_categorical => "Region",
        :ppa_date => "", 
        :ppa_price => "PPA Price (2020 Dollars/MWh)"),
)
for subdf in groupby(ppa_price_long, :region_categorical)
],
Layout(legend_orientation="h",
    legend_x=.9,
        legend_y=1.02,
        legend_xanchor="right",
        legend_yanchor="bottom"))

data: [
  "scatter with fields color, labels, marker, mode, name, type, x, and y",
  "scatter with fields color, labels, marker, mode, name, type, x, and y",
  "scatter with fields color, labels, marker, mode, name, type, x, and y",
  "scatter with fields color, labels, marker, mode, name, type, x, and y",
  "scatter with fields color, labels, marker, mode, name, type, x, and y",
  "scatter with fields color, labels, marker, mode, name, type, x, and y",
  "scatter with fields color, labels, marker, mode, name, type, x, and y",
  "scatter with fields color, labels, marker, mode, name, type, x, and y",
  "scatter with fields color, labels, marker, mode, name, type, x, and y",
  "scatter with fields color, labels, marker, mode, name, type, x, and y"
]

layout: "layout with fields legend, margin, and template"

┌ Warning: Kaleido is not available on this system. Julia will be unable to save images of any plots.
└ @ PlotlyJS /Users/katiepress/.julia/packages/PlotlyJS/4jzLr/src/kaleido.jl:65
┌ Warning: UndefVarError(:artifact_dir)
└ @ PlotlyJS /Users/katiepress/.julia/packages/PlotlyJS/4jzLr/src/kaleido.jl:66
┌ Warning: Kaleido is not available on this system. Julia will be unable to save images of any plots.
└ @ PlotlyJS /Users/katiepress/.julia/packages/PlotlyJS/4jzLr/src/kaleido.jl:65
┌ Warning: UndefVarError(:artifact_dir)
└ @ PlotlyJS /Users/katiepress/.julia/packages/PlotlyJS/4jzLr/src/kaleido.jl:66


Stacked bar in plotly. First need to aggregate the data, which is pretty easy in Julia. I think I'm getting the hang of this syntax because I guessed correctly that grouping by multiple columns required brackets. 

In [86]:
ppa_capacity = combine(groupby(ppa_price_long, [:region_categorical,:ppa_year]), :capacity_mw => sum);

This is the first attempt, looks pretty good for not really having any formatting. 

In [87]:
plot(
    ppa_capacity,
    kind="bar",
    x=:ppa_year,
    y=:capacity_mw_sum,
    color=:region_categorical,
    Layout(barmode="stack")
)

data: [
  "bar with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "bar with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "bar with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "bar with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "bar with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "bar with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "bar with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "bar with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "bar with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "bar with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis"
]

layout: "layout with fields barmode, legend, margin, template, xaxis, and yaxis"

┌ Warning: Kaleido is not available on this system. Julia will be unable to save images of any plots.
└ @ PlotlyJS /Users/katiepress/.julia/packages/PlotlyJS/4jzLr/src/kaleido.jl:65
┌ Warning: UndefVarError(:artifact_dir)
└ @ PlotlyJS /Users/katiepress/.julia/packages/PlotlyJS/4jzLr/src/kaleido.jl:66
┌ Warning: Kaleido is not available on this system. Julia will be unable to save images of any plots.
└ @ PlotlyJS /Users/katiepress/.julia/packages/PlotlyJS/4jzLr/src/kaleido.jl:65
┌ Warning: UndefVarError(:artifact_dir)
└ @ PlotlyJS /Users/katiepress/.julia/packages/PlotlyJS/4jzLr/src/kaleido.jl:66


See if I can get the annotations for the top of the bars in a similar way that I did in R. First, sort the data by year and descending region.

In [88]:
ppa_capacity_year = sort(ppa_capacity, [:ppa_year, order(:region_categorical, rev=true)]);

Now aggregate, thankfully Julia has a first() function just like R. 

In [89]:
ppa_capacity_year = combine(groupby(ppa_capacity_year, :ppa_year),
 :capacity_mw_sum => sum => :capacity_sum,
 :region_categorical => first => :region_categorical);

Convert the capacity_sum to integer so it will look nicer. 

In [90]:
transform!(ppa_capacity_year, :capacity_sum => ByRow(x -> round(Int32, x)) => :capacity_sum);

Now try joining the data. 

In [91]:
ppa_capacity_all = leftjoin(ppa_capacity, ppa_capacity_year, on = [:ppa_year, :region_categorical]);

It worked! The only thing is the colors are not in the same order as the Tableau dashboard, but at this point I think I'm done plotting for the day. 

In [92]:
plot([bar(
    subdf,
    x=:ppa_year,
    y=:capacity_mw_sum,
    color=:region_categorical,
    mode="markers",
    marker_color = mycolorpal[subdf[1, :region_categorical]],
    name=subdf[1, :region_categorical],
    text=:capacity_sum,
    textposition="outside",

)
for subdf in groupby(ppa_capacity_all, :region_categorical)
],
Layout(
        barmode="stack",
        yaxis_title="Capacity (MW-AC)",
        legend_title_text="Region",
        legend_orientation="h",
        legend_x=.9,
        legend_y=1.02,
        legend_xanchor="right",
        legend_yanchor="bottom",
        xaxis_tickvals=[
            2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013,
            2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022]
        ))

data: [
  "bar with fields color, marker, mode, name, text, textposition, type, x, and y",
  "bar with fields color, marker, mode, name, text, textposition, type, x, and y",
  "bar with fields color, marker, mode, name, text, textposition, type, x, and y",
  "bar with fields color, marker, mode, name, text, textposition, type, x, and y",
  "bar with fields color, marker, mode, name, text, textposition, type, x, and y",
  "bar with fields color, marker, mode, name, text, textposition, type, x, and y",
  "bar with fields color, marker, mode, name, text, textposition, type, x, and y",
  "bar with fields color, marker, mode, name, text, textposition, type, x, and y",
  "bar with fields color, marker, mode, name, text, textposition, type, x, and y",
  "bar with fields color, marker, mode, name, text, textposition, type, x, and y"
]

layout: "layout with fields barmode, legend, margin, template, xaxis, and yaxis"

┌ Warning: Kaleido is not available on this system. Julia will be unable to save images of any plots.
└ @ PlotlyJS /Users/katiepress/.julia/packages/PlotlyJS/4jzLr/src/kaleido.jl:65
┌ Warning: UndefVarError(:artifact_dir)
└ @ PlotlyJS /Users/katiepress/.julia/packages/PlotlyJS/4jzLr/src/kaleido.jl:66
┌ Warning: Kaleido is not available on this system. Julia will be unable to save images of any plots.
└ @ PlotlyJS /Users/katiepress/.julia/packages/PlotlyJS/4jzLr/src/kaleido.jl:65
┌ Warning: UndefVarError(:artifact_dir)
└ @ PlotlyJS /Users/katiepress/.julia/packages/PlotlyJS/4jzLr/src/kaleido.jl:66
